# Interactive scatterplot

A basic interactive scatterplot made from JupyterDash + plotly.express
Interactive plots are good for EDA of data with many variables.

In [15]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html, Input, Output
import pandas as pd
import plotly.express as px


In [16]:
df = pd.read_csv('./energy_clean.csv')


In [17]:
vars = df.columns
vars

Index(['DOF Gross Floor Area',
       'Largest Property Use Type - Gross Floor Area (ft²)', 'score',
       'Site EUI (kBtu/ft²)', 'Weather Normalized Site EUI (kBtu/ft²)',
       'Weather Normalized Site Electricity Intensity (kWh/ft²)',
       'Weather Normalized Site Natural Gas Intensity (therms/ft²)',
       'Weather Normalized Source EUI (kBtu/ft²)', 'Natural Gas Use (kBtu)',
       'Weather Normalized Site Natural Gas Use (therms)',
       'Electricity Use - Grid Purchase (kBtu)',
       'Weather Normalized Site Electricity (kWh)',
       'Total GHG Emissions (Metric Tons CO2e)',
       'Direct GHG Emissions (Metric Tons CO2e)',
       'Indirect GHG Emissions (Metric Tons CO2e)',
       'Property GFA - Self-Reported (ft²)',
       'Water Use (All Water Sources) (kgal)',
       'Water Intensity (All Water Sources) (gal/ft²)',
       'Source EUI (kBtu/ft²)', 'Latitude', 'Longitude', 'Community Board',
       'Council District', 'Census Tract'],
      dtype='object')

In [18]:
app = JupyterDash(__name__)

server = app.server 

app.layout = html.Div([
    html.Div([

        html.Div([
            # single-select Drop down widget
            dcc.Dropdown(
                id='xaxis-column', # to be used in callback
                options= [{'label': i, 'value': i} for i in vars], # list of options
                value='Site EUI (kBtu/ft²)' # default value
            ),
            # RadioItems is a component for rendering a set of radio (or option) buttons
            dcc.RadioItems( 
                id='xaxis-type',
                options=['Linear', 'Log'],
                value='Linear',
                inline = True
            )
        ], style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in vars],
                value='score'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                inline = True
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id = 'indicator-graphic')

    
],style = {'backgroundColor': '#111111'})
# component_id or _property can be ommited 
# The variable of the funciton refers to the corresponding property of the id
@app.callback( 
    Output(component_id='indicator-graphic', component_property='figure'), 
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value')
    #, prevent_initial_call=True to prevent callback when refresh/startup
    )
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type): # need to match the order of input
    
    dff = df.dropna(subset = [xaxis_column_name, yaxis_column_name])
    
    fig = px.scatter(x = dff[xaxis_column_name],
                     y = dff[yaxis_column_name])
    # px returns a plotyly.graph_objects.Figure 

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')
    fig.update_layout(
        plot_bgcolor = '#111111', # black plot background color
        paper_bgcolor = '#34F24F' # green surronding color
    )
    
    return fig

In [19]:
app.run_server(mode = 'inline', port = 8051)